# Probabilistic Forecasting with `sktime`

Credits:

### Overview of this notebook

* quick start - probabilistic forecasting
* disambiguation - types of probabilistic forecasts
* details: probabilistic forecasting interfaces
* metrics for, and evaluation of probabilistic forecasts
* advanced composition: pipelines, tuning, reduction
* extender guide


---
### Quick Start - Probabilistic Forecasting with `sktime`

... works exactly like the basic forecasting workflow, replace `predict` by a probabilistic method!

In [ ]:
from sktime.datasets import load_airline
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.base import ForecastingHorizon

# step 1: data specification
y = load_airline()
# step 2: specifying forecasting horizon
fh = [1, 2, 3]
# step 3: specifying the forecasting algorithm
forecaster = ARIMA()
# step 4: fitting the forecaster
forecaster.fit(y, fh=[1,2,3])
# step 5: querying predictions
y_pred = forecaster.predict()

# for probabilistic forecasting:
#   call a probabilistic forecasting method after or instead of step 5
y_pred_int = forecaster.predict_interval(coverage=0.9)
y_pred_int

**probabilistic forecasting methods in `sktime`**:

* forecast intervals    - `predict_interval(fh=None, X=None, coverage=0.90)`
* forecast quantiles    - `predict_quantiles(fh=None, X=None, alpha=[0.05, 0.95])`
* forecast variance     - `predict_var(fh=None, X=None, cov=False)`
* distribution forecast - `predict_proba(fh=None, X=None, marginal=True)`

To check which forecasters in `sktime` support probabilistic forecasting, use the `registry.all_estimators` utility and search for estimators which have the `capability:pred_int` tag (value `True`).

For composites such as pipelines, a positive tag means that logic is implemented if (some or all) components support it.

In [ ]:
from sktime.registry import all_estimators

all_estimators("forecaster", filter_tags={"capability:pred_int": True}, as_dataframe=True)

---
### What is probabilistic forecasting?

todo: insert picture here

#### Intuition

* produce low/high scenarios of forecasts
* quantify uncertainty around forecasts
* produce expected range of variation of forecasts

#### Interface view

**Want** to produce "distribution" or "range" of forecast values,

at time stamps defined by **forecasting horizon** `fh`

given **past data** `y` (series), and possibly exogeneous data `X`

Input, to `fit` or `predict`: `fh`, `y`, `X`

Output, from `predict_probabilistic`: some "distribution" or "range" object

**Big caveat**: there are multiple possible ways to model "distribution" or "range!

Used in practice and easily confused! (and often, practically, confused!)

#### Formal view (endogeneous, one forecast time stamp)

Let  $y(t_1), \dots, y(t_n)$ be observations at fixed time stamps $t_1, \dots, t_n$. 

(we consider $y$ as an $\mathbb{R}^n$-valued random variable)

Let $y'$ be a (true) value, which will be observed at a future time stamp $\tau$.

(we consider $y'$ as an $\mathbb{R}$-valued random variable)

We have the following "types of forecasts" of $y'$:

| Name | param | prediction/estimate of | `sktime` |
| ---- | ----- | ---------------------- | -------- |
| point forecast | | conditional expectation $\mathbb{E}[y'\|y]$ | `predict` |
| variance forecast | | conditional variance $Var[y'\|y]$ | `predict_var` |
| quantile forecast | $\alpha\in (0,1)$ | $\alpha$-quantile of $y'\|y$ | `predict_quantiles` |
| interval forecast | $c\in (0,1)$| $[a,b]$ s.t. $P(a\le y' \le b\| y) = c$ | `predict_interval` |
| distribution forecast | | the law/distribution of $y'\|y$ | `predict_proba` |

Notes:

* different forecasters have different capabilities!
* metrics, evaluation & tuning are different by "type of forecast"
* compositors can "add" type of forecast! Example: bootstrap

##### More formal details & intuition:

* a "point forecast" is a prediction/estimate of the conditional expectation $\mathbb{E}[y'|y]$. Intuition: "out of many repetitions/worlds, this value is the arithmetic average of all observations".
* a "variance forecast" is a prediction/estimate of the conditional expectation $Var[y'|y]$. Intuition: "out of many repetitions/worlds, this value is the average squared distance of the observation to the perfect point forecast".
* a "quantile forecast", at quantile point $\alpha\in (0,1)$ is a prediction/estimate of the $\alpha$-quantile of $y'|y$, i.e., of $F^{-1}_{y'|y}(\alpha)$, where $F^{-1}$ is the (generalized) inverse cdf = quantile function of the random variable y'|y. Intuition: "out of many repetitions/worlds, a fraction of exactly $\alpha$ will have equal or smaller than this value."
* an "interval forecast" or "predictive interval" with (symmetric) coverage $c\in (0,1)$ is a prediction/estimate pair of lower bound $a$ and upper bound $b$ such that $P(a\le y' \le b| y) = c$ and $P(y' \gneq b| y) = P(y' \lneq a| y) = (1 - c) /2$. Intuition: "out of many repetitions/worlds, a fraction of exactly $c$ will be contained in the interval $[a,b]$, and being above is equally likely as being below".
* a "distribution forecast" or "full probabilistic forecast" is a prediction/estimate of the distribution of $y'|y$, e.g., "it's a normal distribution with mean 42 and variance 1". Intuition: exhaustive description of the generating mechanism of many repetitions/worlds.

Notes:

* lower/upper of interval forecasts are quantile forecasts at quantile points $0.5 - c/2$ and $0.5 + c/2$ (as long as forecast distributions are absolutely continuous).
* all other forecasts can be obtained from a full probabilistic forecasts; a full probabilistic forecast can be obtained from all quantile forecasts or all interval forecasts.
* there is no exact relation between the other types of forecasts (point or variance vs quantile)
* in particular, point forecast does not need to be median forecast aka 0.5-quantile forecast. Can be $\alpha$-quantile for any $\alpha$!

Frequent confusion in literature & python packages:
* coverage `c` vs quantile `\alpha`
* coverage `c` vs significance `p = 1-c`
* quantile of lower interval bound, `p/2`, vs `p`
* interval forecasts vs related, but substantially different concepts: confidence interval on predictive mean; Bayesian posterior or credibility interval of the predictive mean
* all forecasts above can be Bayesian, confusion: "posteriors are different" or "have to be evaluted differently"

---
### Probabilistic forecasting interfaces in `sktime` 

This section:

* walkthrough of probabilistic predict methods
* use in update/predict workflow
* multivariate and hierarchical data

All forecasters with tag `capability:pred_int` provide the following:

* forecast intervals    - `predict_interval(fh=None, X=None, coverage=0.90)`
* forecast quantiles    - `predict_quantiles(fh=None, X=None, alpha=[0.05, 0.95])`
* forecast variance     - `predict_var(fh=None, X=None, cov=False)`
* distribution forecast - `predict_proba(fh=None, X=None, marginal=True)`

Generalities:

* methods do not change state, multiple can be called
* `fh` is optional, if passed late
* exogeneous data `X` can be passed

In [ ]:
import numpy as np

from sktime.datasets import load_airline
from sktime.forecasting.theta import ThetaForecaster

# until fit, identical with the simple workflow
y = load_airline()

fh = np.arange(1, 13)

forecaster = ThetaForecaster(sp=12)
forecaster.fit(y, fh=fh)

##### `predict_interval` - interval predictions

Inputs:\
`coverage`, float or list of floats\
nominal coverage(s) of the prediction interval(s) queried

Output: `pandas.DataFrame`\
Row index is fh\
Column has multi-index:\
1st level = variable name from y in fit\
2nd level = coverage fractions in `coverage`\
3rd level = string "lower" or "upper"\

Entries = forecasts of lower/upper interval at nominal coverage in 2nd lvl, for var in 1st lvl

In [ ]:
coverage = 0.9
y_pred_ints = forecaster.predict_interval(coverage=coverage)
y_pred_ints

pretty-plotting the predictive interval forecasts:

In [ ]:
from sktime.utils import plotting

# also requires predictions
y_pred = forecaster.predict()

fig, ax = plotting.plot_series(y, y_pred, labels=["y", "y_pred"])
ax.fill_between(
    ax.get_lines()[-1].get_xdata(),
    y_pred_ints["Coverage"][coverage]["lower"],
    y_pred_ints["Coverage"][coverage]["upper"],
    alpha=0.2,
    color=ax.get_lines()[-1].get_c(),
    label=f"{coverage} cov.pred.intervals",
)
ax.legend();

multiple coverages:

In [ ]:
coverage = [0.5, 0.9, 0.95]
y_pred_ints = forecaster.predict_interval(coverage=coverage)
y_pred_ints

##### `predict_quantiles` - quantile forecasts

Inputs:\
`alpha`, float or list of floats\
quantile points at which quantiles are queried

Output: `pandas.DataFrame`\
Row index is fh\
Column has multi-index:\
1st level = variable name from y in fit\
2nd level = quantile points in `alpha`\

Entries = forecasts of quantiles at quantile point in 2nd lvl, for var in 1st lvl

In [ ]:
alpha = [0.1, 0.25, 0.5, 0.75, 0.9]
y_pred_quantiles = forecaster.predict_quantiles(alpha=alpha)
y_pred_quantiles

pretty-plotting the quantile interval forecasts:

In [ ]:
from sktime.utils import plotting

_, columns = zip(*y_pred_quantiles.iteritems())
fig, ax = plotting.plot_series(y[-50:], *columns)


#### Using probabilistic forecasts with update/predict stream workflow

#

#### Probabilistic forecasting for multivariate and hierarchical data

---
### Metrics for probabilistic forecasts and evaluation


#### overview - theory

Predicted `y` has different form from true `y`, so metrics have form

`metric(y_true: series, y_pred: proba_prediction) -> float`

where `proba_prediction` is the type of the specific "probabilistic prediction type".

I.e., we have the following function signature for a loss/metric $L$:

| Name | param | prediction/estimate of | general form |
| ---- | ----- | ---------------------- | -------- |
| point forecast | | conditional expectation $\mathbb{E}[y'\|y]$ | `metric(y_true, y_pred)` |
| variance forecast | | conditional variance $Var[y'\|y]$ | `metric(y_pred, y_pt, y_var)` (requires point forecast too) |
| quantile forecast | $\alpha\in (0,1)$ | $\alpha$-quantile of $y'\|y$ | `metric(y_true, y_quantiles, alpha)` |
| interval forecast | $c\in (0,1)$| $[a,b]$ s.t. $P(a\le y' \le b\| y) = c$ | `metric(y_true, y_interval, c)` |
| distribution forecast | | the law/distribution of $y'\|y$ | `metric(y_true, y_distribution)` |


#### evaluation of probabilistic forecasters

In [ ]:
from sktime.performance_metrics.forecasting import \
    mean_absolute_percentage_error

mean_absolute_percentage_error(y_test, y_pred, symmetric=False)

---
### Advanced composition: pipelines, tuning, reduction, adding proba forecasts to any estimator


#### Data specification

In [ ]:
from sktime.forecasting.ets import AutoETS

#### Task specification

In [ ]:
# specifying the forecasting horizon: one year ahead, all months
fh = np.arange(1, 13)
fh

In [ ]:
#  in December 1975
# this is the data known in December 1975
y_train = y.loc[:"1957-08"]
y_observed = y_train.copy()
y_observed.tail()

#### Model specification

In [ ]:
# specifying the model
forecaster = AutoETS(auto=True, sp=12, n_jobs=-1)

#### Fitting

In [ ]:
# fit
forecaster.fit(y_train)

#### Prediction

In [ ]:
y_pred

#### Observe new data

In [ ]:
y_observed = y.loc[:"1957-09"]
new_data = y.loc[["1957-09"]]
new_data

#### Update

In [ ]:
forecaster.update(new_data)

#### Predict again

In [ ]:
y_pred

#### Understanding update


In [ ]:
forecaster.update?

### Automating the process

In [ ]:
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from utils import plot_windows

In [ ]:
y = load_airline()
fh = ForecastingHorizon(np.arange(12) + 1)
y_train, y_test = temporal_train_test_split(y, fh=fh)

#### Temporal cross-validation

Check out alternative cross-validation schemes: https://github.com/alan-turing-institute/sktime/blob/main/examples/window_splitters.ipynb

#### Backtesting: Evaluation using temporal cross-validaton

In [ ]:
forecaster = NaiveForecaster(strategy="last", sp=12)
cv = ExpandingWindowSplitter(step_length=12, fh=fh, initial_window=72)
results = evaluate(
    forecaster=forecaster, y=y, cv=cv, strategy="refit", return_data=True
)
results.iloc[:, :5].head()

### Advanced model building & composition

#### Tuning

In [ ]:
from sktime.forecasting.model_selection import (ForecastingGridSearchCV,
                                                SlidingWindowSplitter)

param_grid = {"window_length": [9, 12, 15], "estimator__n_neighbors": np.arange(1, 10)}

regressor = KNeighborsRegressor()
forecaster = make_reduction(regressor, strategy="recursive")

cv = SlidingWindowSplitter(window_length=60, fh=fh)
gscv = ForecastingGridSearchCV(
    forecaster, cv=cv, param_grid=param_grid, strategy="refit"
)

In [ ]:
gscv.best_params_

#### Tuning and AutoML 

In [ ]:
from sktime.forecasting.compose import MultiplexForecaster
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.naive import NaiveForecaster

forecaster = MultiplexForecaster(
    forecasters=[
        ("naive", NaiveForecaster(strategy="last")),
        ("ets", ExponentialSmoothing(trend="add", sp=12)),
    ],
)

forecaster_param_grid = {"selected_forecaster": ["ets", "naive"]}
gscv = ForecastingGridSearchCV(forecaster, cv=cv, param_grid=forecaster_param_grid)

gscv.fit(y_train)
gscv.best_params_

#### Pipelining

In [ ]:
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.transformations.series.detrend import Deseasonalizer, Detrender

In [ ]:
regressor = KNeighborsRegressor()
forecaster = make_reduction(regressor, strategy="recursive")

forecaster = TransformedTargetForecaster(
    [
        ("deseasonalize", Deseasonalizer(sp=12)),
        ("detrend", Detrender()),
        ("forecast", forecaster),
    ]
)

forecaster.fit(y_train)
y_pred = forecaster.predict(fh)

### Pipelining with exogenous data

In [ ]:
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from sktime.datasets import load_macroeconomic
from sktime.forecasting.compose import ForecastingPipeline
from sktime.transformations.series.adapt import TabularToSeriesAdaptor
from sktime.transformations.series.impute import Imputer

In [ ]:
data = load_macroeconomic()
y = data["unemp"]
X = data.drop(columns=["unemp"])

In [ ]:
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X)
fh = ForecastingHorizon(y_test.index, is_relative=False)

In [ ]:
forecaster = ForecastingPipeline(
    steps=[
        ("imputer", Imputer(method="mean")),
        ("scale", TabularToSeriesAdaptor(MinMaxScaler(feature_range=(1, 2)))),
        ("boxcox", TabularToSeriesAdaptor(PowerTransformer(method="box-cox"))),
        ("forecaster", AutoARIMA(suppress_warnings=True)),
    ]
)
forecaster.fit(y=y_train, X=X_train)
y_pred = forecaster.predict(fh=fh, X=X_test)

plot_series(y_train, y_pred, y_test, labels=["y_train", "y_pred", "y_test"])

In [ ]:
from sktime.forecasting.var import VAR

forecaster = VAR()
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
y_pred.head()

---
## Building your own forecaster

Check out our [forecasting extension template](https://github.com/alan-turing-institute/sktime/blob/main/extension_templates/forecasting.py)!

This is a Python file with to-do code blocks that allow you to implement your own, sktime-compatible forecasting algorithm.


---
## Summary

* unified API for univariate & multivariate forecasting 
* integrating other packages (e.g. scikit-learn, statsmodels, pmdarima, prophet)
* composite model building (pipelining, ensembling, tuning, reduction)
* easily extendible

#### Useful resources
* For more details, take a look at [our paper on forecasting with sktime](https://arxiv.org/abs/2005.08067) in which we discuss the forecasting API in more detail and use it to replicate and extend the M4 study.
* For a good introduction to forecasting, see [Hyndman, Rob J., and George Athanasopoulos. Forecasting: principles and practice. OTexts, 2018](https://otexts.com/fpp2/).
* For comparative benchmarking studies/forecasting competitions, see the [M4 competition](https://www.sciencedirect.com/science/article/pii/S0169207019301128) and the [M5 competition](https://www.kaggle.com/c/m5-forecasting-accuracy/overview).
